## 2.10 종합예제
- 스파크 데이터프레임의 스키마 정보를 알아내는 스키마 추론(schema inference)과 파일의 첫 Row를 헤더로 지정하는 옵션을 사용해서 csv파일을 읽음

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Python Spark SQL basic exmaple") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

In [3]:
#.csv("FileStore/tables/data/flight-data/csv/2015-summary.csv")
flightData2015 = spark \
  .read \
  .option("inferschema", "true") \
  .option("header", "true") \
  .csv("/FileStore/tables/data/flight-data/csv/2015_summary-ebaee.csv")

- 스칼라와 파이썬에서 사용하는 데이터프레임은 불특정 다수의 로우와 컬럼을 갖음. 로우의 수를 알 수 없는 이유는 자연 연산형태의 프랜스포메이션이기 때문이며, 스파크는 각 컬럼의 데이터 타입을 추론하기 위해 적은 양의 데이터를 읽음. DataFrame의 take액션을 호출하면 그제야 결과를 확인할 수 있음

In [5]:
display(flightData2015.take(3))

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344


- 정수 데이터 타입인 count컬럼을 기준으로 데이터를 정렬하는 트랜스포메이션을 추가하였음. sort는 트랜스포메이션이기 때문에 데이터에 변화가 일어나지 않지만 explain 메서드를 통해 스파크 쿼리의 실행계획을 확인할 수 있음

In [7]:
# 200개의 셔플 파티션을 생성한 것을 아래에서 확인할 수 있음
display(flightData2015.sort("count").explain())

== Physical Plan ==
Sort [count#871 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#871 ASC NULLS FIRST, 200), [id=#2072]
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#869,ORIGIN_COUNTRY_NAME#870,count#871] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/data/flight-data/csv/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>

- 액션을 호출하면 트랜스포메이션 실행 계획을 시작함. 액션을 시작하기 전 셔플의 출력 파티션 수를 줄임

In [9]:
spark.conf.set("spark.sql.shuffle.partition", "5")
display(flightData2015.sort("count").limit(10))

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
Suriname,United States,1
United States,Estonia,1
United States,Cyprus,1
United States,Croatia,1
Zambia,United States,1
Moldova,United States,1
Burkina Faso,United States,1
United States,Gibraltar,1
Djibouti,United States,1
United States,Singapore,1


### 2.10.1 DataFrame과 SQL
- 스파크 SQL을 사용하면 모든 DataFrame을 테이블이나 뷰(임시 테이블)로 등록한 후 SQL쿼리를 사용할 수 있음. CreateOrReplaceTempView 메서드를 사용함

In [11]:
flightData2015.createOrReplaceTempView("flight_data_2015")

- DataFrame과 SQL 중 간편한 방식으로 트랜스포메이션을 지정할 수 있음. 같은 목적으로 두 종류의 실행 계획을 비교해보면 동일한 기본 실행 계획으로 컴파일된 것을 알 수 있음

In [13]:
query = """
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
"""

sqlWay = spark.sql(query)
dataFrameWay = flightData2015.groupBy('DEST_COUNTRY_NAME').count()

display(sqlWay.explain())

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#869], functions=[finalmerge_count(merge count#932L) AS count(1)#920L])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#869, 200), [id=#2124]
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#869], functions=[partial_count(1) AS count#932L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#869] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/data/flight-data/csv/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

In [14]:
display(dataFrameWay.explain())

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#869], functions=[finalmerge_count(merge count#937L) AS count(1)#927L])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#869, 200), [id=#2163]
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#869], functions=[partial_count(1) AS count#937L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#869] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/data/flight-data/csv/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

- 특정 위치를 왕래하는 최대 비햇 횟수를 구하기 위해 max함수를 활용. max함수는 필터링을 수행해 단일 로우를 결과로 반환하는 트랜스포메이션임

In [16]:
#SQL
spark.sql("SELECT max(count) FROM flight_data_2015").take(1)

Out[14]: [Row(max(count)=370002)]

In [17]:
#python
from pyspark.sql.functions import max
flightData2015.select(max('count')).take(1)

Out[15]: [Row(max(count)=370002)]

- 상위 5개의 도착 국가를 찾아내는 복잡한 코드

In [19]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
""")

display(maxSql)

DEST_COUNTRY_NAME,destination_total
United States,411352
Canada,8399
Mexico,7140
United Kingdom,2025
Japan,1548
Germany,1468
Dominican Republic,1353
South Korea,1048
The Bahamas,955
France,935


In [20]:
from pyspark.sql.functions import desc

display(flightData2015 \
  .groupBy("DEST_COUNTRY_NAME") \
  .sum('count') \
  .withColumnRenamed("sum(count)", "destination_total") \
  .sort(desc("destination_total")))

DEST_COUNTRY_NAME,destination_total
United States,411352
Canada,8399
Mexico,7140
United Kingdom,2025
Japan,1548
Germany,1468
Dominican Republic,1353
South Korea,1048
The Bahamas,955
France,935


In [21]:
display(flightData2015 \
  .groupBy("DEST_COUNTRY_NAME") \
  .sum("count") \
  .withColumnRenamed("sum(count)", "destination_total") \
  .sort(desc('destination_total')) \
  .explain())

== Physical Plan ==
Sort [destination_total#1001L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(destination_total#1001L DESC NULLS LAST, 200), [id=#2403]
 +- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#869], functions=[finalmerge_sum(merge sum#1005L) AS sum(cast(count#871 as bigint))#997L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#869, 200), [id=#2399]
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#869], functions=[partial_sum(cast(count#871 as bigint)) AS sum#1005L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#869,count#871] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/data/flight-data/csv/2015_summary-ebaee.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>

- (1)은 파일을 읽음. 그 다음 (1~2)는 groupBy, sum()을 실행, 마지막으로는 정렬과 limit(5)를 실행하는 순서